In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [2]:
# stocks = pd.read_csv('sp_500_stocks.csv')
# competitor_lst = ["Ticker", "COIN", "HOOD", "MSTR", "SQ", "PYPL", "CME"]
# stocks2 = pd.read_csv(competitor_lst)
# stocks
# stocks3 = pd.read_csv('coin_competitors.csv')
stocks4 = pd.read_csv('coin_competitors_2.csv')
stocks4

,Ticker
0,COIN
1,HOOD
2,MSTR
3,SQ
4,CME


In [3]:
# from APIDEMOPROJECT1 import IEX_CLOUD_API_TOKEN

In [4]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
#API CALL#
# symbol = 'COIN'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
# # print(api_url)
# data = requests.get(api_url).json()
# print(data)

In [6]:
symbol = 'COIN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# print(data.status_code)
print(data)

{'avgTotalVolume': 4096764, 'calculationPrice': 'close', 'change': 6.16, 'changePercent': 0.04279, 'close': 156.14, 'closeSource': 'ffilicoa', 'closeTime': 1671632335972, 'companyName': 'Coinbase Global Inc - Class A', 'currency': 'USD', 'delayedPrice': 155.07, 'delayedPriceTime': 1698903116542, 'extendedChange': -0.93, 'extendedChangePercent': -0.0061, 'extendedPrice': 156.1, 'extendedPriceTime': 1657588252516, 'high': 156.14, 'highSource': 'r  1u tplcneedmye5eaidi', 'highTime': 1699399868098, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 154.072, 'iexCloseTime': 1681904610461, 'iexLastUpdated': 1653855847247, 'iexMarketPercent': 0.03314674536706181, 'iexOpen': 146.32, 'iexOpenTime': 1701920664919, 'iexRealtimePrice': 151.83, 'iexRealtimeSize': 8, 'iexVolume': 97927, 'lastTradeTime': 1728939460228, 'latestPrice': 155.78, 'latestSource': 'Close', 'latestTime': 'April 19, 2022', 'latestUpdate': 1715086688534, 'latestVolume': 2953297, 'low': 147.76, 'l

In [7]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

10.9

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks4['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,COIN,153.20,11.26,N/A
1,HOOD,11.81,-1.55,N/A
2,MSTR,481.19,-8.82,N/A
3,SQ,126.65,392.49,N/A
4,CME,238.40,32.59,N/A


In [10]:
#Removing Glamour Stocks -- FILTER#
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

/Users/school/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,COIN,153.20,11.26,N/A
1,CME,238.40,32.59,N/A
2,SQ,126.65,392.49,N/A


In [11]:
#Deciding how many shares to buy#
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [12]:
portfolio_input()

Enter the value of your portfolio:10000


In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

/Users/school/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,COIN,153.20,11.26,21
1,CME,238.40,32.59,13
2,SQ,126.65,392.49,26


In [14]:
#Building a better & more realistic value strategy#

#NEW SYNTAX: np.NAN -- used as filler for PANDAS module#


symbol = 'COIN'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data["COIN"]["advanced-stats"])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]["advanced-stats"]['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]["advanced-stats"]['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]["advanced-stats"]['enterpriseValue']
ebitda = data[symbol]["advanced-stats"]['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
'''π'''
gross_profit = data[symbol]["advanced-stats"]['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


In [15]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    
]


rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string }&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        enterprise_value = data[symbol]["advanced-stats"]['enterpriseValue']
        ebitda = data[symbol]["advanced-stats"]['EBITDA']
        gross_profit = data[symbol]["advanced-stats"]['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
        pd.Series(
            [
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]["advanced-stats"]['priceToBook'],
            'N/A',
            data[symbol]["advanced-stats"]['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ],
        index = rv_columns
        ),
            ignore_index = True
        )

# rv_dataframe

In [16]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,COIN,155.21,N/A,10.86,N/A,6.11,N/A,4.97,N/A,10.827393,N/A,5.292166,N/A,N/A
1,HOOD,11.94,N/A,-1.58,N/A,1.13,N/A,4.65,N/A,-5.132089,N/A,4.689205,N/A,N/A
2,MSTR,478.28,N/A,-9.01,N/A,4.42,N/A,8.46,N/A,107.758644,N/A,14.895864,N/A,N/A
3,SQ,130.87,N/A,387.04,N/A,15.80,N/A,2.88,N/A,88.390028,N/A,11.309887,N/A,N/A
4,CME,238.70,N/A,32.71,N/A,3.10,N/A,18.34,N/A,25.626138,N/A,17.687717,N/A,N/A


In [17]:
#Dealing with missing data in DrataFrame#
# rv_dataframe[rv_dataframe.isnull().any(axis=1)]
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [18]:
# rv_dataframe.columns

In [19]:
#Calculating value percetniles#

#ONE WAY:#
# metrics = {'Price-to-Earnings Ratio': 'PE Percentile', 'Price-to-Book Ratio': 'PB Percentile', 'Price-to-Sales Ratio': 'PS Percentile', 'EV/EBITDA': 'EV/EBITDA Percentile','EV/GP': 'EV/GP Percentile'}
# for metric in metrics.keys():
#     for row in rv_dataframe.index:
#         rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore( rv_dataframe[metric] , rv_dataframe.loc[row, metric] )


#OTHER WAY:#
from scipy.stats import percentileofscore as score
metrics = {'Price-to-Earnings Ratio': 'PE Percentile', 'Price-to-Book Ratio': 'PB Percentile', 'Price-to-Sales Ratio': 'PS Percentile', 'EV/EBITDA': 'EV/EBITDA Percentile','EV/GP': 'EV/GP Percentile'}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric] , rv_dataframe.loc[row, metric] )



rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,COIN,155.21,N/A,10.86,60.0,6.11,80.0,4.97,60.0,10.827393,40.0,5.292166,40.0,N/A
1,HOOD,11.94,N/A,-1.58,40.0,1.13,20.0,4.65,40.0,-5.132089,20.0,4.689205,20.0,N/A
2,MSTR,478.28,N/A,-9.01,20.0,4.42,60.0,8.46,80.0,107.758644,100.0,14.895864,80.0,N/A
3,SQ,130.87,N/A,387.04,100.0,15.80,100.0,2.88,20.0,88.390028,80.0,11.309887,60.0,N/A
4,CME,238.70,N/A,32.71,80.0,3.10,40.0,18.34,100.0,25.626138,60.0,17.687717,100.0,N/A


In [20]:
#Calculating RV Score#
from statistics import mean

# pe_ratio =
# pb_ratio =
# ps_ratio =
# ev_to_ebitda = 
# ev_to_gross_profit = 


for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
#     print(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,COIN,155.21,N/A,10.86,60.0,6.11,80.0,4.97,60.0,10.827393,40.0,5.292166,40.0,56.0
1,HOOD,11.94,N/A,-1.58,40.0,1.13,20.0,4.65,40.0,-5.132089,20.0,4.689205,20.0,28.0
2,MSTR,478.28,N/A,-9.01,20.0,4.42,60.0,8.46,80.0,107.758644,100.0,14.895864,80.0,68.0
3,SQ,130.87,N/A,387.04,100.0,15.80,100.0,2.88,20.0,88.390028,80.0,11.309887,60.0,72.0
4,CME,238.70,N/A,32.71,80.0,3.10,40.0,18.34,100.0,25.626138,60.0,17.687717,100.0,76.0


In [21]:
#Selecting best value stocks#
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
# rv_dataframe = rv_dataframe[:6]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HOOD,11.94,N/A,-1.58,40.0,1.13,20.0,4.65,40.0,-5.132089,20.0,4.689205,20.0,28.0
1,COIN,155.21,N/A,10.86,60.0,6.11,80.0,4.97,60.0,10.827393,40.0,5.292166,40.0,56.0
2,MSTR,478.28,N/A,-9.01,20.0,4.42,60.0,8.46,80.0,107.758644,100.0,14.895864,80.0,68.0
3,SQ,130.87,N/A,387.04,100.0,15.80,100.0,2.88,20.0,88.390028,80.0,11.309887,60.0,72.0
4,CME,238.70,N/A,32.71,80.0,3.10,40.0,18.34,100.0,25.626138,60.0,17.687717,100.0,76.0


In [22]:
#Calculating number of shares to buy"

portfolio_input()

Enter the value of your portfolio:10000


In [23]:
# print(portfolio_size)
position_size = float(portfolio_size)/len(rv_dataframe.index)
# position_size == 10000/6
for row in rv_dataframe.index:
    rv_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/rv_dataframe.loc[row, "Price"])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HOOD,11.94,167,-1.58,40.0,1.13,20.0,4.65,40.0,-5.132089,20.0,4.689205,20.0,28.0
1,COIN,155.21,12,10.86,60.0,6.11,80.0,4.97,60.0,10.827393,40.0,5.292166,40.0,56.0
2,MSTR,478.28,4,-9.01,20.0,4.42,60.0,8.46,80.0,107.758644,100.0,14.895864,80.0,68.0
3,SQ,130.87,15,387.04,100.0,15.80,100.0,2.88,20.0,88.390028,80.0,11.309887,60.0,72.0
4,CME,238.70,8,32.71,80.0,3.10,40.0,18.34,100.0,25.626138,60.0,17.687717,100.0,76.0


In [ ]:
#Formatting Excel Output#